### quest
- 업무분장(전처리, 모델학습).
- RecurrenceOfSurgery.csv 사용
- 목표변수 범주형 : 척추전방전위증 
   - 
- 설명변수 최소 6개 : 
  - (연속형)
    체중 
    신장
        
  - (범주형)
    환자통증정도,
    스테로이드치료, 
    수술기법
    과거수술횟수
  
- 서비스 대상과 목표 설명, 변수 선택 이유

In [2]:
import pandas as pd

In [11]:
df_ROS = pd.read_csv('../datasets/RecurrenceOfSurgery.csv')
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

In [12]:
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [6]:
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [ ]:
특성 공학 기법

1.Imputation (결측값 처리)

In [24]:
df_ROS_extract = df_ROS[['수술기법', '전방디스크높이(mm)', '후방디스크높이(mm)', '입원기간', '척추전방위증', '환자통증정도', '스테로이드치료' ]]
df_ROS_extract.isnull().sum() 

수술기법           81
전방디스크높이(mm)     0
후방디스크높이(mm)     0
입원기간            0
척추전방위증          0
환자통증정도          0
스테로이드치료         0
dtype: int64

In [25]:
df_ROS_extract_preprocess = df_ROS_extract.dropna()
df_ROS_extract_preprocess[:2]

,수술기법,전방디스크높이(mm),후방디스크높이(mm),입원기간,척추전방위증,환자통증정도,스테로이드치료
0,TELD,16.1,12.3,2,0,10,1
1,TELD,13.7,6.4,1,0,10,1


In [26]:
df_ROS_extract_preprocess.shape

(1813, 7)

#### Scailng & Enconding

In [22]:
df_ROS_extract_preprocess['수술기법'].value_counts()

TELD    1673
IELD     140
Name: 수술기법, dtype: int64

In [16]:
df_ROS_extract_preprocess['전방디스크높이(mm)'].value_counts()

10.00    42
11.10    37
11.30    34
11.20    31
11.70    31
         ..
14.42     1
11.95     1
10.52     1
7.87      1
17.00     1
Name: 전방디스크높이(mm), Length: 273, dtype: int64

In [10]:
df_ROS_extract_preprocess.shape

(1813, 7)

In [17]:
df_ROS_extract_preprocess['후방디스크높이(mm)'].value_counts()

8.20     53
8.80     45
9.30     45
7.60     43
9.40     41
         ..
6.77      1
8.06      1
10.27     1
10.21     1
7.59      1
Name: 후방디스크높이(mm), Length: 218, dtype: int64

In [18]:
df_ROS_extract_preprocess['입원기간'].value_counts()

1     1131
2      459
3      157
4       34
5        6
8        5
0        3
6        3
11       3
7        3
17       2
27       1
46       1
9        1
18       1
16       1
44       1
12       1
Name: 입원기간, dtype: int64

In [19]:
df_ROS_extract_preprocess['척추전방위증'].value_counts()

0    1759
1      54
Name: 척추전방위증, dtype: int64

In [20]:
df_ROS_extract_preprocess['환자통증정도'].value_counts()

7     853
8     386
9     163
2     127
10    107
1      52
3      44
5      40
6      37
4       4
Name: 환자통증정도, dtype: int64

In [21]:
df_ROS_extract_preprocess['스테로이드치료'].value_counts() 

1    1081
0     732
Name: 스테로이드치료, dtype: int64